# mis notas para entender que sucedia

In [ ]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [ ]:
documents_raw[0]

In [ ]:
print(type(documents_raw))

In [ ]:
print(type(documents_raw[0]))

In [ ]:
import json
print(json.dumps(documents_raw, indent=4))

In [ ]:
documents_raw[0]['course']

In [ ]:
documents_raw[0]['documents'][0]

In [ ]:
course_name_to_find = "machine-learning-zoomcamp"
# O para el ejemplo de la imagen:
# course_name_to_find = "mlops-zoomcamp" # si quieres llegar a la pregunta 'Is it going to be live? When?'

# Paso 1: Encontrar el diccionario del curso usando un bucle for
found_course_data = None
for course_dict in documents_raw:
    if course_dict["course"] == course_name_to_find:
        found_course_data = course_dict
        break # Detener la iteración una vez que encontramos el curso

# Paso 2: Acceder a la primera pregunta si el curso fue encontrado
if found_course_data:
    # Asegurarse de que el curso tiene documentos
    if found_course_data["documents"]:
        first_question = found_course_data["documents"][0]["question"]
        print(f"La primera pregunta para '{course_name_to_find}' es: {first_question}")
    else:
        print(f"El curso '{course_name_to_find}' no tiene documentos.")
else:
    print(f"El curso '{course_name_to_find}' no fue encontrado.")

In [ ]:
courses = []

for course in documents_raw:
    courses.append( course['course'])

courses

In [ ]:
documents_raw[0]['documents'][0]

In [ ]:
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
documents

# aqui empieza el código original

In [ ]:
# aqui empi

In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
df.head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


In [4]:
df[df.course == 'data-engineering-zoomcamp'].head()

,course,section,question,text
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
2,data-engineering-zoomcamp,General course-related questions,Course - Can I still join the course after the...,"Yes, even if you don't register, you're still ..."
3,data-engineering-zoomcamp,General course-related questions,Course - I have registered for the Data Engine...,You don't need it. You're accepted. You can al...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...


In [7]:
docs_example = [
    "January course details, register now",
    "Course prerequisites listed in January catalog",
    "Submit January course homework by end of month",
    "Register for January course, no prerequisites",
    "January course setup: Python and Google Cloud"
]

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer()

In [10]:
cv.fit(docs_example)

CountVectorizer()

In [ ]:
names = cv.get_feature_names_out()
names

In [ ]:
X = cv.transform(docs_example)

In [ ]:
X.toarray()

In [ ]:
df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

In [ ]:
cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(docs_example)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
cv = TfidfVectorizer(stop_words='english')
X = cv.fit_transform(docs_example)

names = cv.get_feature_names_out()

df_docs = pd.DataFrame(X.toarray(), columns=names).T
df_docs.round(2)

In [ ]:
query = "Do I need to know python to sign up for the January course?"

In [ ]:
q = cv.transform([query])
q.toarray()

In [ ]:
query_dict = dict(zip(names, q.toarray()[0]))
query_dict

In [ ]:
doc_dict = dict(zip(names, X.toarray()[1]))
doc_dict

In [ ]:
df_qd = pd.DataFrame([query_dict, doc_dict], index=['query', 'doc']).T

In [ ]:
(df_qd['query'] * df_qd['doc']).sum()

In [ ]:
X.dot(q.T).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(X, q)

In [ ]:
df.columns

In [ ]:
fields = ['section', 'question', 'text']
transformers = {}
matrices = {}

for field in fields:
    cv = TfidfVectorizer(stop_words='english', min_df=3)
    X = cv.fit_transform(df[field])

    transformers[field] = cv
    matrices[field] = X

In [ ]:
transformers['text'].get_feature_names_out()

In [ ]:
matrices['text']

In [ ]:
query = "I just singned up. Is it too late to join the course?"

In [ ]:
q = transformers['text'].transform([query])
score = cosine_similarity(matrices['text'], q).flatten()

In [ ]:
mask = (df.course == 'data-engineering-zoomcamp').values
score = score * mask
score[:10]

In [ ]:
import numpy as np

In [ ]:
idx = np.argsort(-score)[:10]
idx

In [ ]:
score[idx]

In [ ]:
df.iloc[idx].text

In [ ]:
fields

In [ ]:
query = "I just signed up. Is it too late to join the course?"

In [ ]:
boost = {'question': 3.0}

score = np.zeros(len(df))

for f in fields:
    b = boost.get(f, 1.0)
    q = transformers[f].transform([query])
    s = cosine_similarity(matrices[f], q).flatten()
    score = score + b * s

In [ ]:
filters = {
    'course': 'data-engineering-zoomcamp'
}

for field, value in filters.items():
    mask = (df[field] == value).values
    score = score * mask

In [ ]:
idx = np.argsort(-score)[:10]
results = df.iloc[idx]
results.to_dict(orient='records')

In [ ]:
class TextSearch:

    def __init__(self, text_fields):
        self.text_fields = text_fields
        self.matrices = {}
        self.vectorizers = {}

    def fit(self, records, vectorizer_params={}):
        self.df = pd.DataFrame(records)

        for f in self.text_fields:
            cv = TfidfVectorizer(**vectorizer_params)
            X = cv.fit_transform(self.df[f])
            self.matrices[f] = X
            self.vectorizers[f] = cv

    def search(self, query, n_results=10, boost={}, filters={}):
        score = np.zeros(len(self.df))

        for f in self.text_fields:
            b = boost.get(f, 1.0)
            q = self.vectorizers[f].transform([query])
            s = cosine_similarity(self.matrices[f], q).flatten()
            score = score + b * s

        for field, value in filters.items():
            mask = (self.df[field] == value).values
            score = score * mask

        idx = np.argsort(-score)[:n_results]
        results = self.df.iloc[idx]
        return results.to_dict(orient='records')

In [ ]:
fields

In [ ]:
index = TextSearch(text_fields=['section', 'question', 'text'])

In [ ]:
index.fit(documents)

In [ ]:
query

In [ ]:
index.search(
    query='I just signed up. Is it too late to join the course?',
    n_results=5,
    boost={'question': 3.0},
    filters={'course': 'data-engineering-zoomcamp'}
)

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
X = matrices['text']
cv = transformers['text']

In [ ]:
svd = TruncatedSVD(n_components=16)
X_emb = svd.fit_transform(X)

In [ ]:
X_emb[0]

In [ ]:
query = 'I just signed up. Is it too late to join the course?'

Q = cv.transform([query])
Q_emb = svd.transform(Q)

In [ ]:
Q_emb[0]

In [ ]:
np.dot(X_emb[0], Q_emb[0])

In [ ]:
score = cosine_similarity(X_emb, Q_emb).flatten()

In [ ]:
idx = np.argsort(-score)[:10]

In [ ]:
list(df.loc[idx].text)

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
nmf = NMF(n_components=16)
X_emb = nmf.fit_transform(X)
X_emb[0]

In [ ]:
Q = cv.transform([query])
Q_emb = nmf.transform(Q)
Q_emb[0]

In [ ]:
score = cosine_similarity(X_emb, Q_emb).flatten()
idx = np.argsort(-score)[:10]
list(df.loc[idx].text)

In [ ]:
import torch
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()  # Set the model to evaluation mode if not training

In [ ]:
texts = [
    "Yes, we will keep all the materials after the course finishes.",
    "You can follow the course at your own pace after it finishes"
]
encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')


In [ ]:
encoded_input

In [ ]:
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**encoded_input)
    hidden_states = outputs.last_hidden_state

In [ ]:
hidden_states.shape

In [ ]:
sentence_embeddings = hidden_states.mean(dim=1)
sentence_embeddings.shape

In [ ]:
sentence_embeddings.numpy()

# note that if use a GPU, first you need to move your tensors to CPU
# sentence_embeddings_cpu = sentence_embeddings.cpu()

In [ ]:
def make_batches(seq, n):
    result = []
    for i in range(0, len(seq), n):
        batch = seq[i:i+n]
        result.append(batch)
    return result

In [ ]:
from tqdm.auto import tqdm

In [ ]:
def compute_embeddings(texts, batch_size=8):
    text_batches = make_batches(texts, 8)
    
    all_embeddings = []
    
    for batch in tqdm(text_batches):
        encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
    
        with torch.no_grad():
            outputs = model(**encoded_input)
            hidden_states = outputs.last_hidden_state
            
            batch_embeddings = hidden_states.mean(dim=1)
            batch_embeddings_np = batch_embeddings.cpu().numpy()
            all_embeddings.append(batch_embeddings_np)
    
    final_embeddings = np.vstack(all_embeddings)
    return final_embeddings

In [ ]:
embeddings = {}

In [ ]:
# fields = ['section', 'question', 'text']

for f in fields:
    print(f'computing embeddings for {f}...')
    embeddings[f] = compute_embeddings(df[f].tolist())

In [ ]:
import pickle

In [ ]:
with open('embeddings.bin', 'wb') as f_out:
    pickle.dump(embeddings)